# Supervisor - Worker Flow for Gen Pod AI Backend

Let us import some basic libraries and load dotenv file the contains necessary API keys.

In [3]:
import os
import pprint as pp
from dotenv import load_dotenv

load_dotenv()

True

## Loading LLM
We will load OpenAI GPT-4 LLM with different temperatures to cater differently in assiting our agents.

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.3)

In [ ]:
code_llm = ChatOpenAI(model="gpt-4-1106-preview",temperature=0.2)

In [25]:
# from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3")

## Import Modules Needed for this Project

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import Document
from typing_extensions import TypedDict
from typing import List
# from langgraph.prebuilt import ToolExecutor, ToolInvocation
import json
# from langchain_core.agents import AgentAction, AgentFinish

## Let us create some Utility function
These function can help later on to read the input files as a json string and create agents that can be later used as nodes in the graph.

In [4]:
# def write_python_file(content, filename):
#     """Writes the given content as a python file to the local directory.

#     Args:
#         content: The string content to write to the file.
#         filename: The filename to save the file as.
#     """
#     # Get the directory of the current script
#     script_dir = os.path.dirname(os.path.realpath(__file__))
    
#     # Construct the full path to the file
#     file_path = os.path.join(script_dir, 'src', f"{filename}.py")
    
#     # Ensure the 'src' directory exists, if not, create it
#     src_dir = os.path.join(script_dir, 'src')
#     if not os.path.exists(src_dir):
#         os.makedirs(src_dir)
    
#     # Write the content to the file
#     with open(file_path, "w") as f:
#         f.write(content)

In [7]:
def read_input_json(file_path) -> str:
    """Reads JSON data from a file and returns it as a string.

    Args:
        file_path: The path to the JSON file.

    Returns:
        A string representation of the JSON data.
    """
    with open(file_path, 'r') as user_input_file:
        data = json.load(user_input_file)
    
    user_input = json.dumps(data)

    return user_input


In [8]:
# To dynamically create any number of agents
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import (
    BaseMessage,
    ToolMessage,
    HumanMessage,
)
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langchain import agents


# def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
#     # Define the prompt using LCEL
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
    
#     # Create the agent using LCEL
#     agent = (
#         {
#             "messages": lambda x: x["messages"],
#             "agent_scratchpad": lambda x: x["agent_scratchpad"],
#         }
#         | prompt
#         | llm
#         | StrOutputParser()
#     )
    
#     return agent
# " If you are unable to fully Complete the task, that's OK, another assistant with different tools "
# " will help where you left off."
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                " You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards completing the task."
                " Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    if not tools:
        prompt = prompt.partial(tool_names="")
        return prompt | llm
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    
    return prompt | llm.bind_tools(tools)

## Define tools to be used by Agents
These tools are custom fuctions that will also go as nodes in the graph and will be called by the agent to take some action.

In [9]:
# Update this path to you local directory where you want to create the project at.
PROJECT_PATH = "/Your/Local/Path/To/The/Directory"

In [10]:
from langchain_core.tools import tool, StructuredTool
from typing import Annotated
import os
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


# @tool("writer-tool", args_schema=WriterInput, return_direct=True)
@tool
def write_generated_code_to_file(
    generated_code: Annotated[str, "The code generated by the agent."],
    file_path: Annotated[str, "The path where the generated code should be written."]
):
# project_folder: Annotated[str, "The name of the project where all files needed for the project are to placed."]
    """
    Writes the provided generated code to the specified file within the project structure.

    Args:
        generated_code (str): The code generated by the agent.
        file_path (str): The path where the generated code should be written.
    """
    try:
        # Prepend PROJECT_PATH to the file_path to ensure all projects are written to the PROJECT_PATH parent folder
        # parent_dir_path= os.path.dirname(PROJECT_PATH)
        # full_path = os.path.join(PROJECT_PATH, file_path)

        # Ensure the directory exists before writing the file
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        # Write the generated code to the file
        with open(file_path, 'w') as file:
            file.write(generated_code)
        
        return f"Successfully wrote generated code to {file_path}."
    except BaseException as e:
        return f"Failed to write generated code. Error: {repr(e)}"
    

In [11]:
import subprocess
from typing import Annotated
from langchain.tools import tool

@tool
def create_git_repo(
    project_name: Annotated[str, "Name of the new Git repository that should be created."]
):
    """
    Creates a new Git repository at the specified path.

    Args:
        project_name (str): Name of the new Git repository that should be created.
    """
    try:
        repo_path = os.path.join(PROJECT_PATH, project_name)
        
        # Ensure the directory exists before initializing the Git repository
        os.makedirs(repo_path, exist_ok=True)
        
        # Initialize a new Git repository
        subprocess.check_output(['git', 'init'], cwd=repo_path)
        
        return f"Successfully created a new Git repository at {repo_path}."
    except BaseException as e:
        return f"Failed to create a new Git repository. Error: {repr(e)}"

In [12]:
import subprocess
from typing import Annotated
from langchain.tools import tool

ALLOWED_COMMANDS = ['mkdir', 'docker', 'python', 'python3', 'pip', 'virtualenv']

@tool
def execute_command(
    command: Annotated[str, "The complete set of commands to be executed on the local machine in order."],
    repo_path: Annotated[str,"Path where the repository is created."]
):
    """
    Executes a command on the local machine.

    Args:
        command (str): The complete set of commands to be executed on the local machine in order.
    """
    # Split the command into parts
    parts = command.split()
    
    # Check if the command is in the whitelist
    if parts[0] not in ALLOWED_COMMANDS:
        return f"Command '{parts[0]}' is not allowed."
    
    try:
        # Execute the command
        # full_path = os.path.join(PROJECT_PATH,repo_path)
        additional_command = f"cd {repo_path} && "
        updated_command = additional_command + command
        result = subprocess.check_output(updated_command, shell=True)
        
        return f"Command executed successfully. Output: {result}"
    except BaseException as e:
        return f"Failed to execute command. Error: {repr(e)}"


## Define State
We first define the state of the graph. This will just be a list of messages, along with a key called 'sender' to track the most recent sender.

In [18]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

## Define Agent Nodes
Using the create_agent utility function let us create different agents that can then be created as a node in the graph.

In [19]:
import functools
from langchain_core.messages import AIMessage
from langchain.agents import AgentType


# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

# architect agent and node
Solution_Architect = create_agent(
    llm,
    [],
    system_message= "You are a requirements specialist. Thoroughly analyze the user provided requirements, decompose it into well defined deliverables and assign these deliverables sequentially to coder. Enforce Standard Project Folder structure.",
)
architect_node = functools.partial(agent_node, agent=Solution_Architect, name="Architect")

# coder_agent = create_agent(
#     llm,
#     [write_generated_code_to_file],
#     system_message="Source files generated by you are written to the local directory",
# )

coder_agent = create_agent(
    code_llm,
    [create_git_repo, execute_command, write_generated_code_to_file],
    system_message="You are an expert Programmer, Follow secure production standards, write well modularized and complete code following standards, Create good project folder structure, write the files to the local filesystem",
)
coder_node = functools.partial(agent_node, agent=coder_agent, name="Coder")

## Define a Tool Node
Next we need to define a node in the graph to run the custom tools that we created earlier.

In [20]:
from langgraph.prebuilt import ToolNode

tools = [create_git_repo, execute_command, write_generated_code_to_file]
tool_node = ToolNode(tools)

## Define Edges
We need to define some logic on what to do next after the agent has its results ready.

In [15]:
# Either agent can decide to end
from typing import Literal


def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

## Need for Persistence
In LangGraph, memory for maintaining context across interactions is facilitated via Checkpointers within StateGraphs.  
1. When setting up a LangGraph workflow, you can ensure state persistence by employing a Checkpointer like `AsyncSqliteSaver`.  
2. Simply include this in your workflow setup by calling `compile(checkpointer=my_checkpointer)` during graph compilation.

In [16]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

## Define the Graph
Let's put all the agent nodes, tool nodes and edge logic defined earlier together to build the workflow. We will also make use of the memory here.

In [21]:
workflow = StateGraph(AgentState)

workflow.add_node("Architect", architect_node)
workflow.add_node("Coder", coder_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Architect",
    router,
    {"continue": "Coder", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "Coder",
    router,
    {"continue": "Architect", "call_tool": "call_tool", "__end__": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Architect": "Architect",
        "Coder": "Coder",
    },
)
workflow.set_entry_point("Architect")

# Lets make sure we pass the memory that we defined earlier to the graph as we compile the workflow.
graph = workflow.compile(checkpointer=memory)

## Invoke the graph
Now that we have a compiled graph with multiple agents and custom tools ready, we can invoke to graph to do a small task for us. 

In [22]:
import pprint as pp

# First lets read in the input that defines the task to be performed by the multi agent setup
user_input = read_input_json("rest_api.json")
pp.pp(user_input)

('{"nodes": [{"id": "node-ee", "name": "user-service", "language": "python", '
 '"restConfig": {"server": {"sqlDB": "MySQL", "port": "3000", "resources": '
 '[{"name": "User", "allowedMethods": ["POST", "LIST", "GET", "PUT", '
 '"DELETE"], "fields": {"Name": {"datatype": "string"}, "City": {"datatype": '
 '"string"}}}]}, "framework": "fastapi"}}]}')


In [25]:
import pprint as pp
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Create this project for me in {PROJECT_PATH}." 
                f"Requirements are {user_input}." 
                "Once you code it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph and the thread ID to be used to persist the memory.
    {
        "recursion_limit": 50,
        "configurable": {"thread_id": "1"}
    },
)
for s in events:
    pp.pp(s)
    print("----")

{'Architect': {'messages': [AIMessage(content="To begin with, let's break down the requirements into well-defined deliverables that the coder can execute sequentially. Here's the decomposition of the requirements:\n\n1. **Project Setup:**\n   - Create a new project directory at `C:/Users/vkumar/Desktop/user-service`.\n   - Inside the `user-service` directory, create a standard project folder structure:\n     - `/app` for application code.\n     - `/app/models` for database models.\n     - `/app/api` for REST API endpoints.\n     - `/app/db` for database configurations.\n     - `/tests` for test cases.\n     - `/docs` for documentation.\n     - `requirements.txt` for Python dependencies.\n     - `main.py` as the entry point of the application.\n\n2. **Environment Setup:**\n   - Initialize a new Python virtual environment within the project directory.\n   - Install FastAPI and any other necessary libraries (e.g., `uvicorn`, `pydantic`, `SQLAlchemy`).\n\n3. **Database Configuration:**\n  

## Agents Say Task Completed but Is it really?
Although the agents marked the task as completed, upon reviewing the task some files created by the agent were empty, thanks to persistence we can now call the supervisor agent to take care of it. Without the memory of the earlier conversation agents wouldn't know what I'm referring to. 

In [26]:
import pprint as pp
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f"user-service project is not implemented fully requirements.txt is missing values, no .gitignore, dockerfile or .dockerignore, test/ folder does not have all the unit test cases" 
                "Once completed coding it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {
        "recursion_limit": 50,
        "configurable": {"thread_id": "1"}
    },
)
for s in events:
    pp.pp(s)
    print("----")

{'Architect': {'messages': [AIMessage(content="Apologies for the oversight. Let's address the missing components:\n\n1. **Update `requirements.txt`:** Populate the `requirements.txt` file with the necessary Python packages.\n\n2. **Create `.gitignore`:** Add a `.gitignore` file to exclude files and directories such as the virtual environment folder (`venv/`) and other non-essential files from the Git repository.\n\n3. **Create `Dockerfile` and `.dockerignore`:** Provide a `Dockerfile` for containerizing the application and a `.dockerignore` file to exclude unnecessary files from the Docker build context.\n\n4. **Complete `tests/` folder:** Write additional unit test cases to cover all the API endpoints.\n\nLet's proceed with these tasks sequentially.\n\nStarting with updating the `requirements.txt` file:\n\n- fastapi\n- uvicorn\n- pydantic\n- sqlalchemy\n- pymysql\n\nProceeding with Coder Task 1: Update `requirements.txt`.", response_metadata={'token_usage': {'completion_tokens': 195, 

# Everything below experimentation

In [39]:
# code_writer = StructuredTool.from_function(
#     func=write_generated_code_to_file,
#     name="code_writer",
#     description="useful for when you need to write a file to the local filesystem",
#     # coroutine= ... <- you can specify an async method if desired as well
# )

<!-- bind_tools() is not available for ollama models, let's try to use ToolExecutor and ToolInvocation -->

In [40]:
# coder_tools = [code_writer]

# tool_executor = ToolExecutor(coder_tools)

In [74]:
# writer_tool = StructuredTool.from_function(write_generated_code_to_file)

In [ ]:
# deployment_specialist = create_agent(
#     llm,
#     [write_generated_code_to_file],
#     system_message="You are an expert Platform Engineer Create files to containerize the project."
# )
# deployment_node = functools.partial(agent_node, agent=deployment_specialist, name="deployer")

## Let's Create Basic chains needed by our agents

In [13]:
# # Requirements Document Generator
# supervisor_prompt = PromptTemplate(
#     template="""You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a detailed, \
#         well defined requirements prompt from it that could be broken down into individual tasks to assign to team members!""", input_variables=["json_user_input"],)

# supervisor_prompt = PromptTemplate(template="""You are excellent at system design and task break down, Break this task {summary} precisely and delegate the deliverables with precise requirements to different {members} to complete it independently.""", input_variables=["summary","members"])

# coder_prompt = PromptTemplate(template="""You are expert python developer, complete this coding {task} following all the security standards, secure packages, error handling, naming conventions and code documentation and then write it to the project folder""", input_variables=["task"])


In [48]:
# Supervisor Chain
members = ["SystemArchitect", "Coder", "Tester", "DeploymentSpecialist"]
supervisor_prompt = PromptTemplate(
    template = """You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a very detailed, \
        well defined requirements prompt from it that could be forwarded to solution architect to break down into individual deliverables.! 
        """,
        input_variables=["json_user_input"],
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

supervisor_chain = supervisor_prompt | llm | StrOutputParser()

json_user_input = read_input_json("test.json")
members_str = ','.join(members)

print(json_user_input)
print(members_str)


result = supervisor_chain.invoke({"json_user_input":json_user_input})

print(result)

{"edges": [], "nodes": [{"id": "node-b0", "name": "student-service", "language": "python", "grpcConfig": {"server": {"sqlDB": "Map", "port": "50052", "resources": [{"fields": {"Name": {"datatype": "str"}, "RollNumber": {"datatype": "int"}, "College": {"datatype": "str"}, "Sign": {"datatype": "str"}, "Marks": {"datatype": "int"}, "GateScore": {"datatype": "int"}, "IsPassed": {"datatype": "bool"}}, "name": "Student"}]}, "framework": "grpcio"}}]}
SystemArchitect,Coder,Tester,DeploymentSpecialist
**Title: Requirement Specification for Student Service Application**

**1. Overview:**
The given JSON text provides a high-level requirement for a service named "student-service". This service will be developed in Python, using the gRPC framework (specifically, `grpcio`). The objective is to create a detailed requirement document that can be used by a solution architect to define the scope of the project and break it down into individual deliverables.

**2. Functional Requirements:**

**2.1 Servic

In [52]:
# Solution Architect Chain
architect_prompt = PromptTemplate(template = """You are excellent at system design and task break down, Break this task {summary} precisely into independent tasks\
                                  and delegate the deliverables with precise requirements to each experts in your team {members_str} to complete it task by task except yourself. 
                                  Ensure that details also include a code snippets that should be used for development of each task so that arguments, parameters, functions names etc can be accessed by all members of the teams. Make sure the response follows a json format for easy parsing""", input_variables=["summary","members_str"]
                                  )

architect_chain = architect_prompt | llm | JsonOutputParser()

summary = result
new_result = architect_chain.invoke({"summary":summary, "members_str":members_str})

print(new_result)

{'title': 'Requirement Specification for Student Service Application', 'overview': "Detailed requirements for the 'student-service' microservice developed in Python using gRPC framework ('grpcio') for a detailed scope and task breakdown.", 'requirements': {'functional': {'service_description': "Develop 'student-service' microservice in Python.", 'grpc_configuration': "Use gRPC framework with 'grpcio', exposing server on port 50052.", 'database_integration': "Interface with an SQL database 'Map'. Design schema and interactions for the 'Student' data model.", 'data_model': {'Student': {'Name': 'string', 'RollNumber': 'integer', 'College': 'string', 'Sign': 'string', 'Marks': 'integer', 'GateScore': 'integer', 'IsPassed': 'boolean'}}, 'api_endpoints': "CRUD endpoints for 'Student' resource. Support complex queries with filtering and sorting."}, 'non_functional': {'performance': 'Optimize for high concurrency.', 'security': 'Implement security best practices, and authentication/authorizati

In [53]:
import pprint as pp
pp.pp(new_result)

{'title': 'Requirement Specification for Student Service Application',
 'overview': "Detailed requirements for the 'student-service' microservice "
             "developed in Python using gRPC framework ('grpcio') for a "
             'detailed scope and task breakdown.',
 'requirements': {'functional': {'service_description': 'Develop '
                                                        "'student-service' "
                                                        'microservice in '
                                                        'Python.',
                                 'grpc_configuration': 'Use gRPC framework '
                                                       "with 'grpcio', "
                                                       'exposing server on '
                                                       'port 50052.',
                                 'database_integration': 'Interface with an '
                                                         "SQL data

In [39]:
# # Construct the options part
# options_str = ", ".join(options)
# print(options_str)

# # Construct the members part
# members_str = ", ".join(members)
# print(members_str)

FINISH, SystemArchitect, Coder, Tester, DeploymentSpecialist
SystemArchitect, Coder, Tester, DeploymentSpecialist


In [56]:
# Coder Chain

coder_task = """{'title': 'Requirement Specification for Student Service Application',
                'overview': "Detailed requirements for the 'student-service' microservice "
                            "developed in Python using gRPC framework ('grpcio') for a "
                            'detailed scope and task breakdown.',
                'requirements': {'functional': {'service_description': 'Develop '
                                                                        "'student-service' "
                                                                        'microservice in '
                                                                        'Python.',
                                                'grpc_configuration': 'Use gRPC framework '
                                                                    "with 'grpcio', "
                                                                    'exposing server on '
                                                                    'port 50052.',
                                                'database_integration': 'Interface with an '
                                                                        "SQL database 'Map'. "
                                                                        'Design schema and '
                                                                        'interactions for the '
                                                                        "'Student' data "
                                                                        'model.',
                                                'data_model': {'Student': {'Name': 'string',
                                                                            'RollNumber': 'integer',
                                                                            'College': 'string',
                                                                            'Sign': 'string',
                                                                            'Marks': 'integer',
                                                                            'GateScore': 'integer',
                                                                            'IsPassed': 'boolean'}},
                                                'api_endpoints': 'CRUD endpoints for '
                                                                "'Student' resource. Support "
                                                                'complex queries with '
                                                                'filtering and sorting.'},
                                'non_functional': {'performance': 'Optimize for high '
                                                                    'concurrency.',
                                                    'security': 'Implement security best '
                                                                'practices, and '
                                                                'authentication/authorization '
                                                                'mechanisms.',
                                                    'scalability': 'Design for scalability.',
                                                    'maintainability': 'Well-documented and '
                                                                        'structured codebase.',
                                                    'testing': 'Comprehensive automated test '
                                                                'suite.'}},
                'deliverables': {'service_implementation': {'description': 'Complete Python '
                                                                            'source code for '
                                                                            "'student-service'. "
                                                                            'Include '
                                                                            'supporting '
                                                                            'scripts/config '
                                                                            'files.',
                                                            'tasks': [{'task_id': 'SI-1',
                                                                        'title': 'Implement '
                                                                                'gRPC Server',
                                                                        'assignee': 'Coder',
                                                                        'code_snippet': 'server '
                                                                                        '= '
                                                                                        'grpc.server(futures.ThreadPoolExecutor(max_workers=10))\n'
                                                                                        "server.add_insecure_port('[::]:50052')"}]},
                                'database_schema': {'description': 'Detailed database schema '
                                                                    "for 'Student' data "
                                                                    'model.',
                                                    'assignee': 'SystemArchitect',
                                                    'code_snippet': 'CREATE TABLE students '
                                                                    '(name TEXT, roll_number '
                                                                    'INTEGER PRIMARY KEY, '
                                                                    'college TEXT, sign '
                                                                    'TEXT, marks INTEGER, '
                                                                    'gatescore INTEGER, '
                                                                    'ispassed BOOLEAN)'},
                                'api_documentation': {'description': 'API documentation with '
                                                                    'example '
                                                                    'requests/responses.',
                                                        'assignee': 'SystemArchitect',
                                                        'code_snippet': 'N/A - Documentation '
                                                                        'task'},
                                'test_suite': {'description': 'Automated test suite for '
                                                                'service functionality.',
                                                'assignee': 'Tester',
                                                'code_snippet': 'class '
                                                                'TestStudentService(unittest.TestCase):\n'
                                                                '    def '
                                                                'test_create_student(self):\n'
                                                                '        # Test creation '
                                                                'logic\n'
                                                                '    # Repeat for read, '
                                                                'update, delete, and query'},
                                'deployment_guide': {'description': 'Step-by-step deployment '
                                                                    'guide.',
                                                    'assignee': 'DeploymentSpecialist',
                                                    'code_snippet': 'N/A - Documentation '
                                                                    'task'},
                                'security_guidelines': {'description': 'Document outlining '
                                                                        'implemented security '
                                                                        'measures.',
                                                        'assignee': 'SystemArchitect',
                                                        'code_snippet': 'N/A - Documentation '
                                                                        'task'}},
                    
                    """

project_structure = """ project_name/
                        │
                        ├── src/
                        │   ├── file1/
                        │   ├── file2/
                        │   └── ...
                        │
                        ├── config/
                        │   ├── config_file1/
                        │   ├── config_file2/
                        │   └── ...
                        │
                        ├── test/
                        │   ├── test_file2/
                        │   ├── test_file2/
                        │   └── ...
                        │
                        ├── utils/
                        │   ├── util1.py
                        │   ├── util2.py
                        │   └── ...
                        │
                        ├── docs/
                        │   ├── doc1.py
                        │   ├── doc2.py
                        │   └── ...
"""
coder_prompt = PromptTemplate(template="""You are expert python developer, complete this coding {task} following all the security standards, secure packages, \
                              error handling, naming conventions and code documentation and give me the complete code as output in json format""", input_variables=["task"]) # following the project structure {structure} while also maintaining crucial metadeta of the files that needs to be created.""", input_variables=["task","structure"])

coder_chain = coder_prompt | llm | JsonOutputParser()

next_result = coder_chain.invoke({"task":coder_task})

pp.pp(next_result)

{'title': 'Requirement Specification for Student Service Application',
 'overview': "Detailed requirements for the 'student-service' microservice "
             "developed in Python using gRPC framework ('grpcio') for a "
             'detailed scope and task breakdown.',
 'requirements': {'functional': {'service_description': 'Develop '
                                                        "'student-service' "
                                                        'microservice in '
                                                        'Python.',
                                 'grpc_configuration': 'Use gRPC framework '
                                                       "with 'grpcio', "
                                                       'exposing server on '
                                                       'port 50052.',
                                 'database_integration': 'Interface with an '
                                                         "SQL data

## Define the Agent State Class that will be passed to other agents in the graph

In [14]:
from typing import TypedDict, List, Union, Dict, Any

class AgentState(TypedDict):
    user_input: str
    current_task: str
    tasks: List[str]
    worker_results: Dict[str, Any]

## Define Supervisor and worker agent node initilazation

In [15]:
def supervisor_agent(state: AgentState, filepath: str) -> AgentState:
    # Initial state setup
    if not state.get('user_input'):
        state['user_input'] = read_input_json(filepath)
        state['current_task'] = 'Convert user input to requirements prompt'
    elif state['current_task'] == 'Convert user input to requirements prompt':
        # Assuming the worker agent has processed the input and returned a requirements prompt
        state['current_task'] = 'Break down requirements into tasks'
    elif state['current_task'] == 'Break down requirements into tasks':
        # Assuming the worker agent has broken down the requirements into tasks
        state['current_task'] = 'Assign tasks to workers'
    return state

In [16]:
def worker_agent(state: AgentState) -> AgentState:
    if state['current_task'] == 'Convert user input to requirements prompt':
        # Simulate converting user input to a requirements prompt
        state['current_task'] = 'Break down requirements into tasks'
        state['tasks'] = ['Task 1', 'Task 2', 'Task 3']
    elif state['current_task'] == 'Break down requirements into tasks':
        # Simulate breaking down the requirements into tasks
        state['current_task'] = 'Assign tasks to workers'
        state['worker_results'] = {'A': ['Task 1'], 'B': ['Task 2'], 'C': ['Task 3']}
    return state

## Now Let's build a simple langgraph and see how this setup works

In [ ]:
from langgraph.graph import StateGraph, END
import json

members = ["SystemArchitect", "Coder", "Tester", "DeploymentSpecialist"]
json_user_input = read_input_json("test.json")
# Initialize the state with default values
initial_state = {
    "user_input": None, # Assuming an empty string as default for user_input
    "current_task": "", # Assuming an empty string as default for current_task
    "tasks": [], # Assuming an empty list for tasks
    "worker_results": {} # Assuming an empty dictionary for worker_results
}


# Create the supervisor agent
supervisor_agent_instance = create_agent(llm, [], f"You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a detailed, \
        well defined requirements prompt from it that could be broken down into individual tasks to assign to team members!")

# Create the worker agent
worker_agent_instance = create_agent(llm, [], "You are an efficient worker who completes the tasks to perforfection.")

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes for the supervisor and worker agents
workflow.add_node("supervisor", supervisor_agent(state=initial_state, filepath = "test.json"))
workflow.add_node("worker", worker_agent)

# Set the entry point as `supervisor`
workflow.set_entry_point("supervisor")

# Add conditional edges to control the flow
workflow.add_conditional_edges(
    "supervisor",
    lambda state: state['current_task'],
    {
        "Convert user input to requirements prompt": "worker",
        "Break down requirements into tasks": "worker",
        "Assign tasks to workers": END
    }
)

# Add an edge from `worker` back to `supervisor`
workflow.add_edge('worker', 'supervisor')

# Compile the graph
app = workflow.compile()

## Let's initialize the AgentState and run the graph to see how it works

In [ ]:
# Invoking the graph to see how it performs.
for s in app.stream(
    initial_state
):
    if "__end__" not in s:
        print(s)
        print("----")